# POC: non-negative matrix factorization
Let's check if non-negative matrix factorization could help us, don't care about the data problems in the data for now


In [ ]:
import altair as alt
import numpy as np
import pandas as pd
from pathlib import Path
import itertools
import datetime
import random
from sklearn.decomposition import NMF
idx = pd.IndexSlice
alt.data_transformers.disable_max_rows()

from util import *
from visualisation import *

In [ ]:
# this reloads code from external modules automatically if it is changed (without having to restart the kernel)
%load_ext autoreload
%autoreload 2

# Util methods

In [ ]:
def vec_dt_replace(series, year=None, month=None, day=None):
    return pd.to_datetime(
        {'year': series.year if year is None else year,
         'month': series.month if month is None else month,
         'day': series.day if day is None else day, 
        'hour': series.hour,
        'minute': series.minute})

## Read the data

In [ ]:
info_df, data_df = read_data(nrows = 1)

## Choose a profile

In [ ]:
profile_to_check = data_df.iloc[0]
profile_matrix = get_profile_matrix(profile_to_check)


## Show the profile

In [ ]:
all_day_chart = alt.Chart(profile_to_check, title = 'All days').mark_line().encode(
    x = 'time:T',
    y = 'value', 
    color = 'date'
)
all_day_chart

## Check barycentric averaging

In [ ]:
average_day_chart = alt.Chart(barycentric_average.to_frame('value').reset_index(), title = 'barycentric average').mark_line().encode(
    x = 'time:T', 
    y = 'value:Q'
)

In [ ]:
(all_day_chart | average_day_chart).resolve_scale(x = 'shared', y= 'shared')

## Check anomaly detection (lof using DTW)

In [ ]:
distance_matrix = get_DTW_distance_matrix(profile_matrix.to_numpy(), 10, 0)
detector = LocalOutlierFactor(n_neighbors = 20, metric = 'precomputed', contamination = 0.1)
labels = detector.fit_predict(distance_matrix)
anomaly_labels = pd.Series(labels == -1, index = profile_matrix.index, name = 'anomaly')


## Cluster using kmedoids

In [ ]:
NB_OF_CLUSTERS = 5

In [ ]:

non_anomalies = profile_matrix[~anomaly_labels]
labels, centers = cluster_timeseries_k_mediods_DTW(non_anomalies.to_numpy(), NB_OF_CLUSTERS, 12, 0)
labels = pd.Series(labels, index = non_anomalies.index, name = 'labels')
centers = pd.DataFrame(centers, columns = non_anomalies.columns)
centers_vis = centers.stack().to_frame('value').reset_index()
profile_vis_cluster = non_anomalies.stack().to_frame('value').join(labels).reset_index()
profile_vis_cluster.time = add_date(profile_vis_cluster.time)

medoid_chart = alt.Chart(centers_vis).mark_line().encode(
    x = 'time:T', 
    y = 'value', 
    color = 'level_0:N'
)
alt.Chart(profile_vis_cluster.reset_index()).mark_line().encode(
    x = 'time:T', 
    y = 'value', 
    color = 'date'
).facet(column = 'labels') 

In [ ]:
profile_vis_cluster1 = (
    profile_vis_cluster
    [['date','labels']]
    .drop_duplicates()
    .assign(
        date = lambda x: pd.to_datetime(x.date),
        week = lambda x: x.date.dt.isocalendar().week, 
        weekday = lambda x: x.date.dt.weekday,
        day = lambda x: x.date.dt.day
    )
)
profile_vis_cluster1

alt.Chart(profile_vis_cluster1).mark_rect().encode(
    x = 'weekday:O', 
    y = 'week:O', 
    color ='labels:N'
)

## Cluster using kmeans and barycentric averaging


In [ ]:
non_anomalies = profile_matrix[~anomaly_labels]
# non_anomalies = profile_matrix
series = SeriesContainer.wrap(non_anomalies.to_numpy())
model = KMeans(k=NB_OF_CLUSTERS, max_it=10, max_dba_it=10, dists_options={"window": 8,'psi':0})
label_dict, performed_it = model.fit(series, use_c=True, use_parallel=True)


labels = pd.Series(index = non_anomalies.index, name = 'labels')
for key,value in label_dict.items(): 
    labels.iloc[list(value)] = key
profile_vis_cluster = non_anomalies.stack().to_frame('value').join(labels).reset_index()
profile_vis_cluster.time = add_date(profile_vis_cluster.time)
alt.Chart(profile_vis_cluster.reset_index()).mark_line().encode(
    x = 'time:T', 
    y = 'value', 
    color = 'date'
).facet(column = 'labels')

In [ ]:
centroids = pd.DataFrame(model.means, columns = add_date(non_anomalies.columns))
centroid_vis = centroids.stack().to_frame('value').reset_index()
bary_chart = alt.Chart(centroid_vis).mark_line().encode(
    x = 'time:T', 
    y= 'value:Q', 
    color = 'level_0:N'
)
bary_chart.properties(title = 'barycenter') | medoid_chart.properties(title = 'medoid')

# decompose using NMF

In [ ]:

# transformed_centers = centers.apply(lambda x: x - np.min(x), axis = 1, raw = True)
matrix = profile_matrix[~anomaly_labels].dropna().to_numpy()
# matrix = transformed_centers.to_numpy()
# alpha controls regularization (pushing weights towards 0 such that representations become sparse)
decomposer = NMF(10, max_iter = 100000, alpha = 0, l1_ratio = 0.9, regularization = 'both', init = 'nndsvd').fit(matrix)
print('reconstruction error', decomposer.reconstruction_err_)
components = decomposer.components_
components_df = pd.DataFrame(components, columns = profile_matrix.columns)
components_df.index.name = 'component_nb'
components_df;

## Show the components

In [ ]:

transformed_centers_vis = profile_matrix.dropna().stack().to_frame('value').reset_index()

component_vis = components_df.stack().to_frame('value').reset_index()
component_vis['time'] = pd.to_datetime(component_vis['time'], format='%H:%M:%S')
component_vis

medoid_chart = alt.Chart(transformed_centers_vis).mark_line().encode(
    x = 'time:T', 
    y = 'value', 
    color = 'level_0:N'
)
alt.Chart(component_vis, title = 'first 5 components').mark_line().encode(
    x = 'time:T', 
    y = 'value:Q', 
    color= 'component_nb:N'
) | medoid_chart

In [ ]:
representation_matrix = pd.DataFrame(decomposer.transform(profile_matrix.dropna()), index = profile_matrix.dropna().index).sort_index()
representation_matrix[0:62].style.background_gradient(cmap = 'Blues', axis = 1)

# show reconstruction + used components

In [ ]:
IDX = 120
transformed = decomposer.transform(profile_matrix.dropna().iloc[[IDX]].to_numpy())
original = decomposer.inverse_transform(transformed)
day = profile_matrix.dropna().iloc[IDX].to_frame('original_value')
day['after_reconstruction'] = original[0]
day = day.stack().reset_index()
day.columns = ['time', 'type', 'value']
day.time = add_date(day.time)
print(transformed)
orig_chart = alt.Chart(day).mark_line().encode(
    x = 'time:T', 
    y = 'value:Q', 
    color = 'type:N'
)

vis_df = components_df.stack().to_frame('value').reset_index()
vis_df['weight'] = transformed[0, vis_df.component_nb]
vis_df.time = add_date(vis_df.time)
vis_df = vis_df[vis_df.weight > 0]
vis_df['value'] = vis_df['value']*vis_df['weight']
vis_df

component_chart = alt.Chart(vis_df).mark_line().encode(
    x = 'time:T', 
    y = 'value', 
    size  = 'weight',
    opacity = 'weight',
    color = 'component_nb:N'
)

orig_chart + component_chart

# Show the reconstruction vs real profile

In [ ]:
IDX = 8
transformed = decomposer.transform(centers.iloc[[IDX]].to_numpy())
original = decomposer.inverse_transform(transformed)
day = centers.iloc[IDX].to_frame('original_value')
day['after_reconstruction'] = original[0]
day = day.stack().reset_index()
day.columns = ['time', 'type', 'value']
day.time = add_date(day.time)
print(transformed)
alt.Chart(day).mark_line().encode(
    x = 'time:T', 
    y = 'value:Q', 
    color = 'type:N'
)